In [166]:
import numpy as np

In [185]:
class DecisionTree:
    def __init__(self, max_depth=None):
        self.root = None
        self.max_depth = max_depth

    def gini_index(self, groups):
        total = sum(len(group) for group in groups)
        gini = 0.0
        for group in groups:
            size = len(group)
            if size == 0:
                continue
            score = 0.0
            labels = [row[-1] for row in group]  # Lấy tất cả nhãn trong nhóm
            unique_labels = set(labels)  # Tạo tập hợp các nhãn duy nhất
            print(f"  - Nhóm có {size} phần tử, nhãn = {labels}")
            for label in unique_labels:
                proportion = labels.count(label) / size  # Tính tỷ lệ nhãn
                print(f"    - Nhãn {label}: Tỷ lệ = {proportion:.4f}")
                score += proportion ** 2  # Cộng vào điểm số Gini
            print(f"    - Điểm Gini cho nhóm này = {1.0 - score:.4f}")
            gini += (1.0 - score) * (size / total)  # Cập nhật Gini cho nhóm này
        return gini

    def split_data(self, dataset, feature_index, threshold):
        left = [row for row in dataset if row[feature_index] <= threshold]
        right = [row for row in dataset if row[feature_index] > threshold]
        # In ra các nhóm sau khi chia
        print(f"    - Dữ liệu chia thành 2 nhóm tại feature {feature_index} với threshold {threshold}:")
        print(f"      - Nhóm trái: {left}")
        print(f"      - Nhóm phải: {right}")
        return left, right

    def best_split(self, dataset):
        class_values = list(set(row[-1] for row in dataset))  # Lấy các giá trị nhãn duy nhất từ dataset
        best_index, best_threshold, best_score, best_groups = None, None, float('inf'), None

        # Duyệt qua tất cả các cột trong dataset (trừ nhãn)
        for index in range(len(dataset[0]) - 1):
            for row in dataset:  # Duyệt từng hàng trong dataset
                groups = self.split_data(dataset, index, row[index])  # Chia dữ liệu theo cột và giá trị
                gini = self.gini_index(groups)  # Tính Gini cho các nhóm
                print(f"    - Split tại feature {index} với threshold {row[index]}: Gini = {gini:.4f}")
                if gini < best_score:  # Nếu Gini nhỏ hơn best_score, cập nhật lại
                    best_index, best_threshold, best_score, best_groups = index, row[index], gini, groups

        print(f"  - Best split: Feature {best_index} tại threshold {best_threshold} với Gini = {best_score:.4f}")
        return best_index, best_threshold, best_groups

    def build_tree(self, dataset, depth=0):
        class_values = [row[-1] for row in dataset]  # Lấy nhãn trong dataset
        if len(set(class_values)) == 1 or (self.max_depth and depth >= self.max_depth):
            return TreeNode(label=max(set(class_values), key=class_values.count))

        feature_index, threshold, groups = self.best_split(dataset)  # Tìm split tốt nhất
        left, right = groups
        if not left or not right:
            return TreeNode(label=max(set(class_values), key=class_values.count))

        print(f"  - Tạo node tại depth {depth}: Feature {feature_index} <= {threshold}")
        left_node = self.build_tree(left, depth + 1)
        right_node = self.build_tree(right, depth + 1)

        return TreeNode(feature_index, threshold, left_node, right_node)

    def fit(self, dataset):
        self.root = self.build_tree(dataset)  # Xây dựng cây

    def print_tree(self, node=None, depth=0):
        if node is None:
            node = self.root
        if node.label is not None:
            print("  " * depth + f"[Leaf] Label: {node.label}")
            return
        print("  " * depth + f"[Node] Feature {node.feature_index} <= {node.threshold}")
        self.print_tree(node.left, depth + 1)
        self.print_tree(node.right, depth + 1)


class TreeNode:
    def __init__(self, feature_index=None, threshold=None, left=None, right=None, label=None):
        self.feature_index = feature_index
        self.threshold = threshold
        self.left = left
        self.right = right
        self.label = label





In [186]:
# Tập dữ liệu đơn giản
dataset = [
    [2.8, 'Yes'],
    [1.2, 'No'],
    [3.6, 'Yes'],
    [4.5, 'No'],
    [5.1, 'Yes']
]

# Khởi tạo và huấn luyện cây quyết định
tree = DecisionTree(max_depth=3)
tree.fit(dataset)

# In ra cây quyết định
print("Cây quyết định được xây dựng:")
tree.print_tree()

    - Dữ liệu chia thành 2 nhóm tại feature 0 với threshold 2.8:
      - Nhóm trái: [[2.8, 'Yes'], [1.2, 'No']]
      - Nhóm phải: [[3.6, 'Yes'], [4.5, 'No'], [5.1, 'Yes']]
  - Nhóm có 2 phần tử, nhãn = ['Yes', 'No']
    - Nhãn No: Tỷ lệ = 0.5000
    - Nhãn Yes: Tỷ lệ = 0.5000
    - Điểm Gini cho nhóm này = 0.5000
  - Nhóm có 3 phần tử, nhãn = ['Yes', 'No', 'Yes']
    - Nhãn No: Tỷ lệ = 0.3333
    - Nhãn Yes: Tỷ lệ = 0.6667
    - Điểm Gini cho nhóm này = 0.4444
    - Split tại feature 0 với threshold 2.8: Gini = 0.4667
    - Dữ liệu chia thành 2 nhóm tại feature 0 với threshold 1.2:
      - Nhóm trái: [[1.2, 'No']]
      - Nhóm phải: [[2.8, 'Yes'], [3.6, 'Yes'], [4.5, 'No'], [5.1, 'Yes']]
  - Nhóm có 1 phần tử, nhãn = ['No']
    - Nhãn No: Tỷ lệ = 1.0000
    - Điểm Gini cho nhóm này = 0.0000
  - Nhóm có 4 phần tử, nhãn = ['Yes', 'Yes', 'No', 'Yes']
    - Nhãn No: Tỷ lệ = 0.2500
    - Nhãn Yes: Tỷ lệ = 0.7500
    - Điểm Gini cho nhóm này = 0.3750
    - Split tại feature 0 với threshold 

In [181]:
def gini_index(dataset):
    total = len(dataset)
    class_values = list(set(row[-1] for row in dataset))  # Lấy các nhãn duy nhất trong dataset

    gini = 1.0
    print(f"  - Tổng số phần tử trong dataset = {total}")
    for class_val in class_values:
        proportion = sum(1 for row in dataset if row[-1] == class_val) / total  # Tính tỷ lệ của mỗi lớp
        print(f"    - Tỷ lệ cho nhãn '{class_val}' = {proportion:.4f}")
        gini -= proportion ** 2  # Cập nhật giá trị Gini
        print(f"    - Gini hiện tại sau khi tính nhãn '{class_val}' = {gini:.4f}")

    return gini

# Dữ liệu mẫu
dataset = [
    [50, 'Yes'],
    [20, 'No'],
    [30, 'No'],
    [70, 'Yes'],
    [40, 'No'],
    [60, 'Yes']
]

# Tính Gini và in ra kết quả chi tiết
gini = gini_index(dataset)

print("Chỉ số Gini của tập dữ liệu là:", gini)


  - Tổng số phần tử trong dataset = 6
    - Tỷ lệ cho nhãn 'No' = 0.5000
    - Gini hiện tại sau khi tính nhãn 'No' = 0.7500
    - Tỷ lệ cho nhãn 'Yes' = 0.5000
    - Gini hiện tại sau khi tính nhãn 'Yes' = 0.5000
Chỉ số Gini của tập dữ liệu là: 0.5


In [184]:

dataset2 = [
    [50, 700, 'Yes'],
    [20, 400, 'No'],
    [30, 500, 'No'],
    [70, 800, 'Yes'],
    [40, 600, 'No'],
    [60, 750, 'Yes'],
]

tree = DecisionTree(max_depth=3)
tree.fit(dataset)

print("Cây quyết định được xây dựng:")
tree.print_tree()


    - Dữ liệu chia thành 2 nhóm tại feature 0 với threshold 50:
      - Nhóm trái: [[50, 'Yes'], [20, 'No'], [30, 'No'], [40, 'No']]
      - Nhóm phải: [[70, 'Yes'], [60, 'Yes']]
  - Nhóm có 4 phần tử, nhãn = ['Yes', 'No', 'No', 'No']
    - Nhãn No: Tỷ lệ = 0.7500
    - Nhãn Yes: Tỷ lệ = 0.2500
    - Điểm Gini cho nhóm này = 0.3750
  - Nhóm có 2 phần tử, nhãn = ['Yes', 'Yes']
    - Nhãn Yes: Tỷ lệ = 1.0000
    - Điểm Gini cho nhóm này = 0.0000
    - Split tại feature 0 với threshold 50: Gini = 0.2500
    - Dữ liệu chia thành 2 nhóm tại feature 0 với threshold 20:
      - Nhóm trái: [[20, 'No']]
      - Nhóm phải: [[50, 'Yes'], [30, 'No'], [70, 'Yes'], [40, 'No'], [60, 'Yes']]
  - Nhóm có 1 phần tử, nhãn = ['No']
    - Nhãn No: Tỷ lệ = 1.0000
    - Điểm Gini cho nhóm này = 0.0000
  - Nhóm có 5 phần tử, nhãn = ['Yes', 'No', 'Yes', 'No', 'Yes']
    - Nhãn No: Tỷ lệ = 0.4000
    - Nhãn Yes: Tỷ lệ = 0.6000
    - Điểm Gini cho nhóm này = 0.4800
    - Split tại feature 0 với threshold 20: Gin